In [1]:
import folium
import ipywidgets as widgets
import math
from folium.map import Marker, Layer
from folium.features import PolyLine
from IPython.display import display, update_display

In [ ]:
class UpdatableMap(object):
    
    def __init__(self):
        
        #BOULDER!
        self._current_location = [40.0, -105.0]
        self._saved_locations = []
        self._old_markers = []
        self._old_lines = []
        
        self._map = folium.Map(self._current_location, max_bounds=True)
    
        self._current_marker = Marker(location=self._map.location, 
                                      popup=folium.Popup(str(self._map.location[0]) 
                                                         + ", " 
                                                         + str(self._map.location[1]))
                                     , icon=folium.Icon(color='green'))
        self._current_marker.add_to(self._map)
    
        
        display(self._map, display_id="main_map")

    def set_current_location(self, location, redraw_map=False):
        self._current_location = location
        if redraw_map:
            self._redraw_map()
    
    def add_saved_location(self, location):
        self._saved_locations.append(location)
        self._redraw_map()
    
    def remove_saved_location(self, location):
        self._saved_locations.remove(location)
        self._redraw_map()
    
    def get_current_location(self):
        return self._current_location
        
    def get_map(self):
        return self._map
    
    def get_saved_locations(self):
        return self._saved_locations
    
    def _redraw_map(self):
        
        #We can't remove the old markers or lines using Folium, 
        #but we can break it and assign non-real coordinates and
        #make them dissappear into space
        for marker in self._old_markers:
            marker.location = [1000, 1000]
        for line in self._old_lines:
            line.location = [[1000, 1000], [1000, 1000]]

        self._current_marker.location = self._current_location
        self._current_marker.popup = folium.Popup(str(self._current_location[0]) 
                                                         + ", " 
                                                         + str(self._current_location[1]))
            
        for saved_location in self._saved_locations:
            marker = Marker(location=saved_location, 
                                      popup=folium.Popup(str(saved_location[0]) 
                                                         + ", " 
                                                         + str(saved_location[1])))
            marker.add_to(self._map)
            self._old_markers.append(marker)

        if len(self._saved_locations) == 2:
            line = PolyLine(self._saved_locations)
            line.add_to(self._map)
            self._old_lines.append(line)
        
        if len(self._saved_locations) > 2:
            sorted_locations = self._sort_locations(self._saved_locations)
            sorted_locations.append(sorted_locations[0])
            line = PolyLine(sorted_locations)
            line.add_to(self._map)
            self._old_lines.append(line)

        self._set_fit_bounds()
        update_display(self._map, display_id="main_map")
        
    def _set_fit_bounds(self):
        
        ne_location_lat = self._current_location[0]
        ne_location_long = self._current_location[1]
        sw_location_lat = self._current_location[0]
        sw_location_long = self._current_location[1]
        
        for saved_location in self._saved_locations:   
            if saved_location[0] < sw_location_lat:
                sw_location_lat = saved_location[0]
            if saved_location[1] < sw_location_long:
                sw_location_long = saved_location[1]
           
        for saved_location in self._saved_locations:
            if saved_location[0] > ne_location_lat:
                ne_location_lat = saved_location[0]
            if saved_location[1] > ne_location_long:
                ne_location_long = saved_location[1]
                
        sw_location = [sw_location_lat, sw_location_long]
        ne_location = [ne_location_lat, ne_location_long]

        self._map.fit_bounds([sw_location, ne_location])    
        
    def _polar_sort(self, locations):
        lat, lon = zip(*((location[0], location[1]) for location in locations))
        ave_lat = float(sum(lat))/len(lat)
        ave_lon = float(sum(lon))/len(lon)
        return sorted(locations, key=lambda location: math.atan2(location[0]-ave_lat, location[1]-ave_lon))
        
    def _sort_locations(self, locations):
        lat,lon = zip(*((location[0], location[1]) for location in self._polar_sort(locations)))
        sorted_locations = []
        for i in range(0, len(lat)):
            sorted_locations.append([lat[i], lon[i]])
        return sorted_locations

In [78]:
class SpatialSearchSelector(object):

    def __init__(self, updatable_map):
        self._updatable_map = updatable_map
        self._current_location = self._updatable_map.get_current_location()
        self._layout_ui()
        self._initialize_ui()
        display(self._container)
    
    def _on_submit_button_clicked(self, button):
        print("GO GET DATA!")
        
    def _on_update_button_clicked(self, button):
        self._updatable_map.set_current_location([float(self._lat_box.value), float(self._lon_box.value)], True)

    def _on_add_button_clicked(self, button):
        self._updatable_map.set_current_location([float(self._lat_box.value), float(self._lon_box.value)])
        new_option = self._lat_box.value + ", " + self._lon_box.value
        if new_option not in self._location_select.options:
            self._location_select.options = self._location_select.options + (new_option,)
            self._location_select.value = new_option
            self._updatable_map.add_saved_location([float(self._lat_box.value), float(self._lon_box.value)])
            self._updatable_map.set_current_location([float(self._lat_box.value), float(self._lon_box.value)])
    
    def _on_remove_button_clicked(self, button):
        self._updatable_map.set_current_location([float(self._lat_box.value), float(self._lon_box.value)])
        old_options_list = list(self._location_select.options)
        
        if self._location_select.value is not None:
            old_options_list.remove(self._location_select.value)
            
            location_string_array = self._location_select.value.split(",")
            
            self._updatable_map.remove_saved_location([float(location_string_array[0].strip()), 
                                                        float(location_string_array[1].strip())])
            
            if old_options_list:
                self._location_select.options = tuple(old_options_list)
                self._location_select.value = self._location_select.options[0]
                
            else:
                self._location_select.options = ()
                self._location_select.value = None
      
    def _initialize_ui(self):
        self._lat_box.value = str(self._current_location[0])
        self._lon_box.value = str(self._current_location[1])
    
    def _layout_ui(self):
        
        #Labels
        self._lat_label = widgets.Label(
            value = 'Current Latitude: ', 
            layout = widgets.Layout(
                width = '35%', 
                margin = '5px 5px 0px 5px'
            )
        )
        self._lon_label = widgets.Label(
            value='Current Longtitude: ', 
            layout = widgets.Layout(
                width = '35%', 
                margin = '5px 5px 0px 5px'
            )
        )
        self._location_label = widgets.Label(
            value='List of Selected Locations: ', 
            layout = widgets.Layout(
                width = '100%', 
                margin = '5px 5px 0px 5px'
            )
        )
        
        #Textfields
        self._lat_box = widgets.Text(
            layout = widgets.Layout(
                width = '90%', 
                margin = '0px 5px 0px 5px'
            )
        )
        self._lon_box = widgets.Text(
            layout = widgets.Layout(
                width = '90%', 
                margin = '0px 5px 0px 5px'
            )
        )
        
        #Buttons
        self._update_button = widgets.Button(
            description = 'Update location on Map',
            layout = widgets.Layout(
                width = '33%', 
                margin = '15px 5px 0px 5px'
            )
        )
        self._update_button.on_click(self._on_update_button_clicked)
        
        self._add_button = widgets.Button(
            description = 'Add location to List',
            layout = widgets.Layout(
                width = '33%', 
                margin = '15px 5px 5px 5px'
            )
        )
        self._add_button.on_click(self._on_add_button_clicked)
        
        self._remove_button = widgets.Button(
            description = 'Remove location from List',
            layout = widgets.Layout(
                width = '34%', 
                margin = '15px 5px 5px 5px'
            )
        )
        self._remove_button.on_click(self._on_remove_button_clicked)
        
        self._submit_button = widgets.Button(
            description = 'Submit Search to Geodex.org',
            layout = widgets.Layout(
                width = '99%', 
                margin = '5px 5px 5px 5px'
                
            )
        )
        self._submit_button.on_click(self._on_submit_button_clicked)
        
        #Selects
        self._location_select = widgets.Select(
            layout = widgets.Layout(
                width = '100%', 
                height = '100%', 
                margin = '5px 0px 5px 0px'
            )
        )
        
        #Containers
        self._location_container = widgets.VBox(
            [self._lat_label, 
             self._lat_box, 
             self._lon_label, 
             self._lon_box]
        )
        self._location_container.layout.width = '50%'
        
        self._location_select_container = widgets.VBox(
            [self._location_label, 
             self._location_select]
        )
        self._location_select_container.layout.width = '50%'
        
        self._middle_container = widgets.HBox(
            [self._location_container, 
             self._location_select_container]
        )
        self._middle_container.layout.width = '100%'
        
        self._bottom_container = widgets.HBox(
            [self._update_button, 
             self._add_button, 
             self._remove_button]
        )
        
        self._container = widgets.VBox(
            [self._middle_container, 
             self._bottom_container, 
             self._submit_button]
        )

In [81]:
updatable_map = UpdatableMap()
selector = SpatialSearchSelector(updatable_map)

A Jupyter Widget